# Analysis of Safecast radiation data

In [87]:
import pandas as pd
import plotly.express as px
#from scripts.preprocessing.process_measurements import *

## Data loading

### Load the dataset

In [88]:
data = pd.read_csv('data/measurements_daily.csv')

df = pd.DataFrame(data)
df

,Device ID,Unit,Latitude,Longitude,Height,Measurement Day,Average Value
0,26.0,usv,-48.980217,69.433594,NaN,1969-12-31,1.73
1,26.0,usv,18.975000,72.825800,NaN,1969-12-31,0.59
2,26.0,usv,19.993998,73.861084,NaN,1969-12-31,0.00
3,26.0,usv,19.616718,74.234619,NaN,1969-12-31,0.04
4,26.0,usv,20.004322,73.921509,NaN,1969-12-31,0.16
...,...,...,...,...,...,...,...
568706,100309.0,status,37.217980,141.000970,9.0,2024-04-13,29.91
568707,100321.0,cpm,37.752099,140.470826,60.0,2024-04-13,39.21
568708,100322.0,cpm,37.752099,140.470826,60.0,2024-04-13,0.05
568709,100139.0,status,37.495123,140.995519,94.0,2024-04-13,23.45


## Data preprocessing

### Drop incorrect measurements

In [89]:
df['Unit'].value_counts()

cpm             424447
celcius          86095
status           51182
usv               5337
PM10 ug/m3         735
DeviceType2        618
microsievert       187
DeviceType1         34
HUMD%               20
1                   15
 cpm                10
CPM                  8
NOXppm               8
Cpm                  5
0                    5
pm2.5                3
RSSI                 1
PM1                  1
Name: Unit, dtype: int64

#### Incorrect unit names

We see that most measurements have the unit "cpm". The notation does not have the same case, so we can substitute corresponding units that are stored in different formats (` cpm`, `CPM` and `Cpm` is the same as `cpm`).

Also replace `microsivert` with `usv` as these are the same units.

In [90]:
# substitute ` cpm`, `CPM` and `Cpm` units with `cpm`
df['Unit'] = df['Unit'].str.replace(' cpm', 'cpm')
df['Unit'] = df['Unit'].str.replace('CPM', 'cpm')
df['Unit'] = df['Unit'].str.replace('Cpm', 'cpm')

# substitute micorsivert with usv
df['Unit'] = df['Unit'].str.replace('microsievert', 'usv')

df['Unit'].value_counts()

cpm            424470
celcius         86095
status          51182
usv              5524
PM10 ug/m3        735
DeviceType2       618
DeviceType1        34
HUMD%              20
1                  15
NOXppm              8
0                   5
pm2.5               3
PM1                 1
RSSI                1
Name: Unit, dtype: int64

In [91]:
# Drop all row that use units with counts less than 100
df = df[df['Unit'].map(df['Unit'].value_counts()) > 100]

df['Unit'].value_counts()

cpm            424470
celcius         86095
status          51182
usv              5524
PM10 ug/m3        735
DeviceType2       618
Name: Unit, dtype: int64

#### Measurements with `DeviceType2` unit

Because measurements for the `DeviceType2` unit always have the same value and we don't know what the actual unit actually is, we can drop these measurements as well.

In [92]:
df[df['Unit'] == 'DeviceType2']['Average Value'].describe()

count    618.0
mean     130.0
std        0.0
min      130.0
25%      130.0
50%      130.0
75%      130.0
max      130.0
Name: Average Value, dtype: float64

In [93]:
df = df[df['Unit'] != 'DeviceType2']

#### Measurements with `celcius` unit

Drop all measurements with the `celcius` unit as this unit is used to measure temperature, not radiation. (The unit should be named `celsius`, though)

In [94]:
df[df['Unit'] == 'celcius']['Average Value'].describe()

count    86095.000000
mean        19.269490
std         20.194046
min       -503.320000
25%         12.920000
50%         18.220000
75%         26.440000
max       1103.470000
Name: Average Value, dtype: float64

In [95]:
df = df[df['Unit'] != 'celcius']

#### Measurements with `status` unit

Let's see what we get for the `status` unit

In [96]:
df[df['Unit'] == 'status']['Average Value'].describe()

count    51182.000000
mean        11.663770
std        113.631285
min      -1000.000000
25%         20.520000
50%         25.570000
75%         29.110000
max       3149.770000
Name: Average Value, dtype: float64

And see how it compares to `usv` and `cpm` units. If the range is not even close to either of them, we will drop the `status` unit as it cannot be easily converted to the correct unit used for radiation measurement.

In [97]:
df[df['Unit'] == 'usv']['Average Value'].describe()

count    5524.000000
mean        0.791226
std         4.756369
min         0.000000
25%         0.080000
50%         0.092556
75%         0.130000
max       115.000000
Name: Average Value, dtype: float64

In [98]:
df[df['Unit'] == 'cpm']['Average Value'].describe()

count    4.244700e+05
mean     2.134603e+11
std      1.390894e+14
min     -8.777799e+12
25%      1.707000e+01
50%      3.002000e+01
75%      3.900000e+01
max      9.061864e+16
Name: Average Value, dtype: float64

Since the order of magnitude of average values with the `status` differs much from both `usv` and `cpm`, we will drop all of these measurements as well.

In [99]:
df = df[df['Unit'] != 'status']

#### Measurements with `PM10 ug/m3` unit

This unit is used to measure air pollution, not radiation, so we can safely drop all rows with this unit as well.

In [100]:
df = df[df['Unit'] != 'PM10 ug/m3']

### Standarize the measurement unit

Show all units after dropping incorrect measurements

In [101]:
df['Unit'].value_counts()

cpm    424470
usv      5524
Name: Unit, dtype: int64

We have 2 units left. We will convert the `cpm` unit to `usv`

In [102]:
def convert_to_cpm(row):
    if row['Unit'] == 'usv':
        # Assuming 1 µSv/h = 200 CPM
        return row['Average Value'] * 200
    elif row['Unit'] == 'cpm':
        return row['Average Value']

df['Average Value'] = df.apply(convert_to_cpm, axis=1)

# Since all data is now in CPM, we can drop the 'Unit' column
df.drop('Unit', axis=1, inplace=True)

df

,Device ID,Latitude,Longitude,Height,Measurement Day,Average Value
0,26.0,-48.980217,69.433594,NaN,1969-12-31,346.00
1,26.0,18.975000,72.825800,NaN,1969-12-31,118.00
2,26.0,19.993998,73.861084,NaN,1969-12-31,0.00
3,26.0,19.616718,74.234619,NaN,1969-12-31,8.00
4,26.0,20.004322,73.921509,NaN,1969-12-31,32.00
...,...,...,...,...,...,...
568704,100301.0,37.217980,141.000970,9.0,2024-04-13,40.27
568705,100302.0,37.217980,141.000970,9.0,2024-04-13,13.41
568707,100321.0,37.752099,140.470826,60.0,2024-04-13,39.21
568708,100322.0,37.752099,140.470826,60.0,2024-04-13,0.05


In [103]:
df['Average Value'].describe()

count    4.299940e+05
mean     2.107181e+11
std      1.381931e+14
min     -8.777799e+12
25%      1.700000e+01
50%      3.000000e+01
75%      3.900000e+01
max      9.061864e+16
Name: Average Value, dtype: float64

### Drop outliers

We can convert all `float64` types to `float32` for faster calculations

In [104]:
df.dtypes

Device ID          float64
Latitude           float64
Longitude          float64
Height             float64
Measurement Day     object
Average Value      float64
dtype: object

In [105]:
df['Device ID'] = df['Device ID'].astype('uint8')

to_float32 = ['Latitude', 'Longitude', 'Height', 'Average Value']
df[to_float32] = df[to_float32].astype('float32')

df['Measurement Day'] = pd.to_datetime(df['Measurement Day'])

df.dtypes

Device ID                   uint8
Latitude                  float32
Longitude                 float32
Height                    float32
Measurement Day    datetime64[ns]
Average Value             float32
dtype: object

## Data visualization

In [106]:
# Get average sensor measurements in the last year (Measurement day column is the day of the measurement, use all dates between 2024-01-01 and 2024-04-01)
df_2024 = df[(df['Measurement Day'] >= '2024-01-01') & (df['Measurement Day'] <= '2024-04-01')]
# Now average for the same location
df_2024 = df_2024.groupby(['Latitude', 'Longitude']).mean().reset_index()

df_2024

,Latitude,Longitude,Device ID,Height,Average Value
0,-34.955399,138.625793,249.0,NaN,29.000000
1,0.000000,0.000000,8.0,0.0,4.100000
2,22.284821,114.139999,157.0,NaN,73.803856
3,27.826099,-82.628899,224.0,NaN,12.724483
4,31.833193,130.301926,125.5,13.0,20.720636
...,...,...,...,...,...
1401,51.961971,5.858359,237.0,NaN,31.920000
1402,51.961971,5.858360,237.0,NaN,31.792000
1403,51.980701,9.234500,108.0,NaN,15.754745
1404,52.427601,4.971100,205.0,NaN,21.521219


In [107]:
fig = px.scatter_geo(
    df_2024, 
    lat='Latitude', 
    lon='Longitude', 
    color='Average Value',
    title='Radiation levels',
    color_continuous_scale=['green', 'yellow', 'red', 'purple'],
)
fig.update_layout(height=800)
fig.show()

In [108]:
import requests
import numpy as np
from urllib.request import urlopen
from json.decoder import JSONDecodeError

def get_elevation(lat, long):
    try:
        query = f'https://api.open-elevation.com/api/v1/lookup?locations={lat},{long}'
        response = requests.get(query).json()
        elevation = response[0]['elevation']
        return elevation
    except (KeyError, IndexError):
        return None
    except JSONDecodeError:
        print("Error: Unable to decode JSON response. The API might be down or returning unexpected data.")
        return None

def fill_missing_height(df):
    missing_height_rows = df[pd.isnull(df['Height'])]
    for index, row in missing_height_rows.iterrows():
        lat, lon = row['Latitude'], row['Longitude']
        height = get_elevation(lat = lat, long = lon)
        if height is not None:
            df.at[index, 'Height'] = height


In [116]:
import pandas as pd
import random
import plotly.graph_objects as go

df = df[df['Average Value'] >= 0]
df['Measurement Day'] = pd.to_datetime(df['Measurement Day'], unit='s')

grouped = df.groupby(['Device ID', 'Measurement Day'])['Average Value'].mean().reset_index()
fig = go.Figure()

random_device_id = random.sample(list(grouped['Device ID'].unique()), 10)

for device_id in random_device_id:
    device_data = grouped[grouped['Device ID'] == device_id]
    device_id_str = str(device_id)
    fig.add_trace(go.Scatter(x=device_data['Measurement Day'], y=device_data['Average Value'], mode='lines', name=device_id_str))

fig.update_layout(
    title='Average Value on Measurement Day per Device ID',
    xaxis=dict(title='Measurement Day'),
    yaxis=dict(title='Average Value', type='log', range=[0, None]),
    hovermode='x',
    showlegend=True,
    margin=dict(l=0, r=0, t=30, b=0),
    height=600
)

fig.show()
